In [14]:
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.0.0-beta1


In [2]:
def make_toy_dataset(num_data):
    X = np.random.randn(num_data, 3)
    y = 3 * X[:, 0] - 2 * X[:, 1]**3 + 2 * X[:, 2]**2 + 0.5 * np.random.randn(num_data)
    y = y[:, np.newaxis]
    return X, y

In [3]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.Dense(10, activation="relu", input_dim=3),
        keras.layers.Dense(10, activation="relu"),
        keras.layers.Dense(1)
    ])

    return model

In [46]:
N_train = 20000
N_test = 4000

num_epochs = 15
learning_rate = 0.01
batch_size = 64

In [47]:
# get toy datasets
X_train, y_train = make_toy_dataset(N_train)
X_test, y_test = make_toy_dataset(N_test)

# make datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=512).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(100)

In [48]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()

start = time.time()

for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = loss_fn(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))

end = time.time()
print(end - start, "(sec)")

epoch 1: loss=17.115, val_loss=3.375274, mae=2.029, val_mae=0.992
epoch 2: loss=5.456, val_loss=2.238686, mae=1.199, val_mae=0.890
epoch 3: loss=3.136, val_loss=1.774873, mae=1.010, val_mae=0.835
epoch 4: loss=2.144, val_loss=1.472416, mae=0.889, val_mae=0.779
epoch 5: loss=1.624, val_loss=1.407722, mae=0.814, val_mae=0.759
epoch 6: loss=1.381, val_loss=1.452342, mae=0.769, val_mae=0.781
epoch 7: loss=1.240, val_loss=1.224803, mae=0.731, val_mae=0.720
epoch 8: loss=1.143, val_loss=1.063762, mae=0.706, val_mae=0.674
epoch 9: loss=1.081, val_loss=0.930714, mae=0.688, val_mae=0.635
epoch 10: loss=1.053, val_loss=0.846581, mae=0.677, val_mae=0.612
epoch 11: loss=0.994, val_loss=0.817619, mae=0.663, val_mae=0.603
epoch 12: loss=0.983, val_loss=0.796920, mae=0.655, val_mae=0.585
epoch 13: loss=0.943, val_loss=0.776129, mae=0.645, val_mae=0.580
epoch 14: loss=0.909, val_loss=0.752312, mae=0.636, val_mae=0.573
epoch 15: loss=0.871, val_loss=0.738428, mae=0.624, val_mae=0.571
30.362996339797974

In [49]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


@tf.function
def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

epoch 1: loss=17.350, val_loss=3.608866, mae=2.032, val_mae=1.004
epoch 2: loss=6.333, val_loss=2.454494, mae=1.211, val_mae=0.910
epoch 3: loss=3.939, val_loss=1.827998, mae=1.040, val_mae=0.828
epoch 4: loss=2.751, val_loss=1.508731, mae=0.934, val_mae=0.760
epoch 5: loss=2.305, val_loss=1.303485, mae=0.877, val_mae=0.724
epoch 6: loss=1.914, val_loss=1.155705, mae=0.821, val_mae=0.692
epoch 7: loss=1.659, val_loss=1.033534, mae=0.777, val_mae=0.664
epoch 8: loss=1.436, val_loss=0.947385, mae=0.738, val_mae=0.638
epoch 9: loss=1.278, val_loss=0.895112, mae=0.708, val_mae=0.618
epoch 10: loss=1.189, val_loss=0.884551, mae=0.689, val_mae=0.615
epoch 11: loss=1.072, val_loss=0.869028, mae=0.668, val_mae=0.609
epoch 12: loss=0.985, val_loss=0.864501, mae=0.651, val_mae=0.606
epoch 13: loss=0.931, val_loss=0.835071, mae=0.637, val_mae=0.596
epoch 14: loss=0.895, val_loss=0.804948, mae=0.628, val_mae=0.584
epoch 15: loss=0.867, val_loss=0.776026, mae=0.620, val_mae=0.574
12.359346628189087

In [50]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


@tf.function
def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def train_epoch(train_dataset):
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)


@tf.function
def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
@tf.function
def eval_epoch(test_dataset):
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)

    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    train_epoch(train_dataset)
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    eval_epoch(test_dataset)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

epoch 1: loss=17.949, val_loss=3.986042, mae=2.112, val_mae=1.071
epoch 2: loss=6.944, val_loss=3.429074, mae=1.286, val_mae=1.090
epoch 3: loss=5.643, val_loss=2.928119, mae=1.193, val_mae=1.055
epoch 4: loss=2.933, val_loss=1.601469, mae=0.950, val_mae=0.787
epoch 5: loss=2.298, val_loss=1.300984, mae=0.861, val_mae=0.732
epoch 6: loss=1.907, val_loss=1.136901, mae=0.818, val_mae=0.723
epoch 7: loss=1.615, val_loss=1.193632, mae=0.789, val_mae=0.745
epoch 8: loss=1.486, val_loss=1.134603, mae=0.768, val_mae=0.730
epoch 9: loss=1.401, val_loss=1.108949, mae=0.754, val_mae=0.727
epoch 10: loss=1.339, val_loss=1.061932, mae=0.739, val_mae=0.714
epoch 11: loss=1.299, val_loss=1.020469, mae=0.730, val_mae=0.704
epoch 12: loss=1.257, val_loss=0.975169, mae=0.717, val_mae=0.692
epoch 13: loss=1.188, val_loss=0.946663, mae=0.706, val_mae=0.687
epoch 14: loss=1.149, val_loss=0.896120, mae=0.698, val_mae=0.670
epoch 15: loss=1.107, val_loss=0.871174, mae=0.688, val_mae=0.659
5.655078649520874 